In [1]:
import swissgrid
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import holidays
from sklearn.linear_model import LinearRegression

In [2]:
ts = swissgrid.grid_data.iloc[:,0].fillna(method='bfill')
SAMPS_DAY = 24*4

In [3]:
weather = pd.read_csv('data/weather_data_filtered.csv')
weather['timestamp'] = pd.to_datetime(weather['utc_timestamp'])
weather = weather[['timestamp', 'CH_temperature']].set_index('timestamp').sort_index()
weather = weather.resample('1 H').first()
weather = weather.tz_localize(None)
weather = weather.resample('15 T').interpolate('linear')

In [4]:
# align the two sequences
ts = ts[:weather.index[-1]]
weather = weather[ts.index[0]:]

In [5]:
days = ts.index
# special days
def get_holy(prov):
    h = holidays.Switzerland(prov=prov)
    return [day in h for day in days]
holidays_cantons = pd.DataFrame(data = {prov:get_holy(prov) for prov in holidays.Switzerland.PROVINCES},
             index = days)
#add sundays (hurts MSE)
# holidays_cantons = (holidays_cantons.T | (holidays_cantons.index.dayofweek == 6)).T

cantons = pd.read_csv('data/cantons.csv').set_index('Code')
#magic trick
cantons['Population'] = cantons['Population'].str.extract('([^\[]*)').iloc[:,0].str.split(',').str.join('').astype(np.intp)

holiday_pop = holidays_cantons*cantons['Population']

### autoregressive part

The consumption $L_{hd}$ today at day $d$ and hour $h$ depends on $L_{h-1d}$, $L_{hd-1}$, $L_{hd-7}$

In [6]:
def build_ts_regressor(ts, fourier_order=4):
    #interact annual pattern like (6)
    week_lag_idx = np.arange(ts.size-SAMPS_DAY*7)
    t = ((
        ts.index[SAMPS_DAY*7:ts.size] - pd.datetime(year=ts.index[0].year, month=1, day=1)
    ).total_seconds()//(60*24*60/SAMPS_DAY)).astype(np.intp)
    samps_year = SAMPS_DAY*7*52
    sins =  np.array([np.sin(2*q*np.pi*(t/samps_year)) for q in range(1, fourier_order+1)]).T
    cosins =np.array([np.cos(2*q*np.pi*(t/samps_year)) for q in range(1, fourier_order+1)]).T
    week_lagged_reg = ts.values[week_lag_idx,None]*np.hstack([np.ones((week_lag_idx.size,1)), sins, cosins])
    
    #interact daily indicators with daily lag (5)
    week_day = ts.index[SAMPS_DAY*7:ts.size].dayofweek
    I = np.eye(7)
    week_day_indicators = np.array([I[i] for i in week_day])
    day_lag_idx = np.arange(SAMPS_DAY*6, ts.size-SAMPS_DAY)
    day_lagged_reg = ts.values[day_lag_idx,None]*week_day_indicators
    
    # hourly lag regressor (7)
    hour_lag_idx = np.arange(SAMPS_DAY*7 - 1, ts.size-1)
    hour_lagged_reg = ts.values[hour_lag_idx][:,None]
    
    #temperature
    weather_reg = weather.values[SAMPS_DAY*7:ts.size]
    
    #special days
    holiday_reg = holiday_pop.sum(axis=1).values[SAMPS_DAY*7:ts.size, None]
    holiday_lagged_reg = holiday_pop.sum(axis=1).values[:ts.size - SAMPS_DAY*7,None]
    
    return np.hstack(
        [
            week_lagged_reg,
            day_lagged_reg,
            hour_lagged_reg,
            weather_reg,
            holiday_reg,
            holiday_lagged_reg
        ]
    )

In [7]:
reg = build_ts_regressor(ts)
# lr = LinearRegression().fit(reg, ts.values[7*SAMPS_DAY:])

In [8]:
import statsmodels.api as sm

In [10]:
mod = sm.OLS(ts.values[7*SAMPS_DAY:], reg)
res = mod.fit()

In [11]:
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 5.744e+07
Date:                Mon, 03 Jun 2019   Prob (F-statistic):               0.00
Time:                        21:39:38   Log-Likelihood:            -3.2412e+06
No. Observations:              279835   AIC:                         6.482e+06
Df Residuals:                  279815   BIC:                         6.483e+06
Df Model:                          20                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             0.0637      0.001    126.311      0.000       0.063       0.065
x2            -0.0027   5.55e-05    -49.257      0.000      -0.003      -0.003
x3          1.128e-05   4.32e-05      0.261      0.794   -7.34e-05    9.59e-05
x4             0.0002   4.29e-05      4.240      0.000    9.78e-05       0.000
x5             0.0002   4.22e-05      3.937      0.000    8.34e-05       0.000
x6            -0.0034   8.38e-05    -40.314      0.000      -0.004      -0.003
x7          8.862e-06   4.25e-05      0.208      0.835   -7.45e-05    9.22e-05
x8             0.0002   4.32e-05      4.573      0.000       0.000       0.000
x9            -0.0006   4.26e-05    -13.080      0.000      -0.001      -0.000
x10            0.0363      0.000     73.233      0.000       0.035       0.037
x11            0.0302      0.000     73.334      0.000       0.029       0.031
x12            0.0296      0.000     73.273      0.000       0.029       0.030
x13            0.0295      0.000     73.350      0.000       0.029       0.030
x14            0.0293      0.000     73.465      0.000       0.028       0.030
x15            0.0258      0.000     73.028      0.000       0.025       0.026
x16            0.0278      0.000     73.502      0.000       0.027       0.029
x17            0.9099      0.001   1600.969      0.000       0.909       0.911
x18         -595.4113     11.870    -50.161      0.000    -618.676    -572.146
x19           -0.0024   3.92e-05    -60.627      0.000      -0.002      -0.002
x20            0.0021   3.93e-05     53.753      0.000       0.002       0.002
==============================================================================
Omnibus:                     2479.383   Durbin-Watson:                   1.085
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             2582.538
Skew:                           0.219   Prob(JB):                         0.00
Kurtosis:                       3.172   Cond. No.                     6.02e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 6.02e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [118]:
MAPE_triv = np.mean(np.abs((ts.values[7*SAMPS_DAY-1:-1] - ts.values[7*SAMPS_DAY:]))/ts.values[7*SAMPS_DAY:])
MAPE_triv

0.013428251107830377

In [14]:
MAPE = np.mean(np.abs((res.predict(reg) - ts.values[7*SAMPS_DAY:]))/ts.values[7*SAMPS_DAY:])
MAPE

0.012889730248821443

### Moving Average part

We can now add a moving average component to the mix and see if our forecasting imporves. The moving average takes into account previous hour errors and previous week error.
The estimation is done by applying the ordinary linear model, and treating residuals as estimated innovations, and iterating the least square estimation, and innovation estimations.

In [15]:
reg = build_ts_regressor(ts)
eps = np.zeros_like(ts.values[7*SAMPS_DAY:])
# we drop an additional leading week so the least squares estimation is done
# with all the estimated epsilons, previous week before included.
for c in range(30):
    eps_reg = np.hstack([eps[:-7*SAMPS_DAY,None], eps[6*SAMPS_DAY:-SAMPS_DAY,None]])
    mod = sm.OLS(exog=np.hstack([reg[7*SAMPS_DAY:],eps_reg]), endog=ts.values[14*SAMPS_DAY:])
    res = mod.fit()
    eps = ts.values[7*SAMPS_DAY:] - res.predict(np.hstack([reg, np.vstack([np.zeros([7*SAMPS_DAY,eps_reg.shape[1]]), eps_reg])]))
    print(c, np.mean(np.abs(eps[7*SAMPS_DAY:]/ts.values[14*SAMPS_DAY:])))

0 0.012891331041182683
1 0.006111266152547367
2 0.012204949234665176
3 0.007349233560088345
4 0.011653519642260943
5 0.008100415360184365
6 0.011247288427572057
7 0.008535613591909425
8 0.010994334348652945
9 0.00879621386684479
10 0.0108397184426317
11 0.008959235732781849
12 0.010722813642729869
13 0.009094050269061262
14 0.010618749471466123
15 0.009224237783588152
16 0.010516031483327972
17 0.009345019998851564
18 0.010419877623092945
19 0.009449423842067923
20 0.010336393871360908
21 0.009532049704080715
22 0.010268287147299578
23 0.009595802317127455
24 0.0102164300068617
25 0.009646227591525561
26 0.010172747477543294
27 0.009687317717169417
28 0.010135573150687001
29 0.009722403174537259
30 0.010104606843200139
31 0.009753236236578304
32 0.01007674929078985
33 0.009779669951064026
34 0.010052395481345351
35 0.009803158257182277
36 0.010032054370681151
37 0.009822608543991865
38 0.01001465879078973
39 0.009838909388565017
40 0.01000017768294308
41 0.00985299458977985
42 0.009988

KeyboardInterrupt: 

In [16]:
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 8.693e+07
Date:                Mon, 03 Jun 2019   Prob (F-statistic):               0.00
Time:                        21:44:32   Log-Likelihood:            -3.1618e+06
No. Observations:              279163   AIC:                         6.324e+06
Df Residuals:                  279141   BIC:                         6.324e+06
Df Model:                          22                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             0.0411      0.000    101.944      0.000       0.040       0.042
x2            -0.0017   4.31e-05    -40.564      0.000      -0.002      -0.002
x3          3.692e-05   3.34e-05      1.104      0.269   -2.86e-05       0.000
x4          8.563e-05   3.32e-05      2.578      0.010    2.05e-05       0.000
x5           8.67e-05   3.27e-05      2.654      0.008    2.27e-05       0.000
x6            -0.0020    6.5e-05    -30.914      0.000      -0.002      -0.002
x7         -1.728e-05   3.29e-05     -0.524      0.600   -8.18e-05    4.73e-05
x8          8.429e-05   3.35e-05      2.518      0.012    1.87e-05       0.000
x9            -0.0004   3.31e-05    -11.386      0.000      -0.000      -0.000
x10            0.0222      0.000     57.135      0.000       0.021       0.023
x11            0.0182      0.000     56.241      0.000       0.018       0.019
x12            0.0180      0.000     56.943      0.000       0.017       0.019
x13            0.0179      0.000     57.001      0.000       0.017       0.019
x14            0.0178      0.000     56.945      0.000       0.017       0.018
x15            0.0155      0.000     56.220      0.000       0.015       0.016
x16            0.0170      0.000     57.203      0.000       0.016       0.018
x17            0.9428      0.000   2099.063      0.000       0.942       0.944
x18         -359.3557      9.225    -38.954      0.000    -377.437    -341.275
x19           -0.0016   3.04e-05    -53.616      0.000      -0.002      -0.002
x20            0.0015   3.08e-05     47.380      0.000       0.001       0.002
x21            0.5111      0.003    202.641      0.000       0.506       0.516
x22            0.4196      0.002    168.105      0.000       0.415       0.425
==============================================================================
Omnibus:                     3410.758   Durbin-Watson:                   1.538
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             4178.395
Skew:                           0.202   Prob(JB):                         0.00
Kurtosis:                       3.442   Cond. No.                     6.03e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 6.03e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

## Multiple Equations

By changing the single regression into 96 individual parameter sets, one for each daily time, we are back to the multiple equations setting described in Clements, Hurn and Li.

In [28]:
reg = build_ts_regressor(ts)
eps = np.zeros_like(ts.values[7*SAMPS_DAY:])
regressions = [None for i in range(SAMPS_DAY)]

for c in range(25):
    eps_reg = np.hstack([eps[:-7*SAMPS_DAY,None], eps[6*SAMPS_DAY:-SAMPS_DAY,None]])
    for daily_time in range(SAMPS_DAY):
        regressions[daily_time] = sm.OLS(
                exog=np.hstack(
                    [reg[7*SAMPS_DAY+daily_time::SAMPS_DAY],eps_reg[daily_time::SAMPS_DAY]]
                ),
                endog=ts.values[daily_time + 14*SAMPS_DAY::SAMPS_DAY]
            ).fit()
        eps[daily_time::SAMPS_DAY] = ts.values[daily_time + 7*SAMPS_DAY::SAMPS_DAY]\
            - regressions[daily_time].predict(
                np.hstack([
                        reg[daily_time::SAMPS_DAY],
                        np.vstack([np.zeros([7*SAMPS_DAY,eps_reg.shape[1]]), eps_reg])[daily_time::SAMPS_DAY]
                ])
            )
    if c % 10 == 0:
        print(c, np.mean(np.abs(eps[7*SAMPS_DAY:]/ts.values[14*SAMPS_DAY:])))

0 0.004463099639687451
10 0.0043431666931691305
20 0.0043424927367846184
30 0.004342451186844226
40 0.0043424485292942305


KeyboardInterrupt: 

In [36]:
regressions[48].conf_int().shape

(22, 2)

In [188]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels import api as sm

In [46]:
for c_i in range(0,22):
    lb = [regressions[i].conf_int()[c_i, 0] for i in range(SAMPS_DAY)]
    ub = [regressions[i].conf_int()[c_i, 1] for i in range(SAMPS_DAY)]
#     plt.fill_between(np.arange(SAMPS_DAY), lb,ub)
#     plt.hlines(0, 0, 95)
#     plt.show()
# uncomment for plot of evolution of coefficients along the day

# this plot is not similar at all to that of the paper


In [223]:
#acf should be more or less flat?

# plot_acf(eps, lags=np.arange(0,22));